In [0]:
print("Healthy")

In [0]:
from sqlalchemy import create_engine
import json
with open('/Workspace/Credentials/db_data.json', 'r') as fp:
    data = json.load(fp)

host = data['redshift']['host']
user = data['redshift']['user']
passwd = data['redshift']['passwd']
database = data['redshift']['database']

conn = create_engine(f"postgresql+psycopg2://{user}:{passwd}@{host}:5439/{database}")

def load_table(schema_name, table_name, dataframe):
    chunk_size = 10000
    num_rows = len(dataframe)

    conn = create_engine(f"postgresql+psycopg2://{user}:{passwd}@{host}:5439/{database}")
    num_iterations = (num_rows // chunk_size) + 1 if num_rows % chunk_size != 0 else num_rows // chunk_size

    for i in range(num_iterations):
        start_idx = i * chunk_size
        end_idx = (i + 1) * chunk_size if (i + 1) * chunk_size < num_rows else num_rows

        chunk_df = dataframe.iloc[start_idx:end_idx]

        chunk_df.to_sql(schema= schema_name, name = table_name, con =conn, index = False, if_exists = 'append', chunksize = 5000, method = 'multi')

    #print("Data loaded successfully.")
    

In [0]:
from datetime import datetime, timedelta
print("Started: " + datetime.today().strftime('%Y-%m-%d %H:%M:%S'))

# Loop through each table and transfer data from MySQL to MSSQL
import time
while True:

    from re import search
    from sqlalchemy import create_engine
    import duckdb

    import pandas as pd 
    import numpy as np 
    import mysql.connector

    pd.set_option('display.max_columns', None)

    import warnings
    warnings.filterwarnings('ignore')


    MifosDB = mysql.connector.connect(
      host=data['mifos']['host'],
      user=data['mifos']['user'],
      passwd=data['mifos']['passwd'],
      database = data['mifos']['database']
    )



    conn = create_engine(f"postgresql+psycopg2://{user}:{passwd}@{host}:5439/{database}")
    pd.set_option('display.float_format', lambda x: '%.2f' % x)



    #Address Info
    lastr = pd.read_sql_query(f'''select max(id) as ltr, max(load_date) as last_date from mifostenantprod_.m_address_info ''', conn)
    ltr = (lastr['ltr'][0], lastr['last_date'][0])
    #print(ltr[0], '-', ltr[1])    
    query = f"SELECT * FROM `Address Info` where id > {ltr[0]} ORDER BY id LIMIT 500000"
    df = pd.read_sql(query, con=MifosDB)
    df['load_date'] = datetime.today().strftime('%Y-%m-%d %H:%M:%S')
    print(len(df), f" of table `Address Info`")
    load_table('mifostenantprod_', 'm_address_info', df)
    #print(ltr[0], "Ended: " + datetime.today().strftime('%Y-%m-%d %H:%M:%S'))
    #print('\n\n')


    #Other Bank Details
    lastr = pd.read_sql_query(f'''select max(client_id) as ltr, max(load_date) as last_date from mifostenantprod_.m_other_bank_details ''', conn)
    ltr = (lastr['ltr'][0], lastr['last_date'][0])
    #print(ltr[0], '-', ltr[1]) 
    query = f"SELECT * FROM `Other Bank Details` WHERE client_id > {ltr[0]} ORDER BY client_id LIMIT 500000"
    df = pd.read_sql(query, con=MifosDB)
    df['load_date'] = datetime.today().strftime('%Y-%m-%d %H:%M:%S')
    print(len(df), f" of table `Other Bank Details`")
    load_table('mifostenantprod_', 'm_other_bank_details', df)
    #print(ltr[0], "Ended: " + datetime.today().strftime('%Y-%m-%d %H:%M:%S'))
    #print('\n\n')        


    #m_transaction_request
    lastr = pd.read_sql_query(f'''select max(id) as ltr, max(load_date) as last_date from mifostenantprod_.m_transaction_request ''', conn)
    ltr = (lastr['ltr'][0], lastr['last_date'][0])
    #print(ltr[0], '-', ltr[1]) 
    query = f"""
            SELECT 
            id, 
            del_flag, 
            m.deleted_on, 
            m.version, 
            m.category, 
            left(m.image_tag, 10000) AS image_tag, 
            m.latitude, 
            m.longitude, 
            LEFT(m.note_image, 10000) AS note_image,
            LEFT(m.remarks, 10000) AS remarks,
            LEFT(m.transaction_brand_name, 10000) AS transaction_brand_name,
            m.transaction_id
            FROM m_transaction_request m
            where id > {ltr[0]}
            ORDER BY id LIMIT  500000;
            """
    df = pd.read_sql(query, con=MifosDB)
    df['load_date'] = datetime.today().strftime('%Y-%m-%d %H:%M:%S')
    print(len(df), f" of table m_transaction_request")
    load_table('mifostenantprod_', 'm_transaction_request', df)
    #print(ltr[0], "Ended: " + datetime.today().strftime('%Y-%m-%d %H:%M:%S'))
    #print('\n\n')       



    # Close database connections
    MifosDB.close()

time.sleep(30) #In Seconds